In [81]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
import sys
sys.path.append("..")

In [83]:
import os
import subprocess
import pickle as pkl
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
import argparse # Para construir o objeto args
from tqdm.notebook import tqdm
import librosa
import torch
import sys # Para adicionar caminhos
import shutil # Para copiar/mover arquivos temporários, se necessário

In [84]:
# GOOGLE COLAB

try:
    # Tenta detectar se está no Colab para definir um caminho base diferente
    import google.colab
    IN_COLAB = True
    BASE_PROJECT_PATH = "/content/Anomaly_Detection_MFCC_TCC/" # Exemplo se clonado no Colab
    # Monte seu Google Drive se os dados/código estiverem lá
    # from google.colab import drive
    # drive.mount('/content/drive')
    # BASE_PROJECT_PATH = "/content/drive/MyDrive/TCC/Anomaly_Detection_MFCC_TCC/"
    # !git clone https://github.com/Igor-C-Assuncao/Anomaly_Detection_MFCC_TCC.git $BASE_PROJECT_PATH 
except:
    IN_COLAB = False
    BASE_PROJECT_PATH = "C:\\Users\\igorc\\Desktop\\Implementação TCC"  # SEU CAMINHO LOCAL AQUI


In [85]:
# Adicionar caminhos dos seus módulos ao sys.path para importação
sys.path.append(os.path.abspath(os.path.join(BASE_PROJECT_PATH))) 
sys.path.append(os.path.abspath(os.path.join(BASE_PROJECT_PATH, "mtsa")))
sys.path.append(os.path.abspath(os.path.join(BASE_PROJECT_PATH, "WAE")))


In [86]:
# ----- Importações dos seus Módulos (APÓS ADICIONAR AO SYS.PATH) -----
from mtsa.utils import files_train_test_split, Wav2Array
from mtsa.features.mel import Array2Mfcc

from WAE import ArgumentParser # Assume que ArgumentParser.py está em WAE/ e define parse_arguments()
from WAE.models.TCN_AAE import Encoder_TCN, Decoder_TCN, LSTMDiscriminator_TCN
from WAE.models.LSTM_AAE import Encoder as LSTMEncoder, Decoder as LSTMDecoder, SimpleDiscriminator # etc.



In [87]:
MIMII_DATA_PATH = os.path.join(BASE_PROJECT_PATH, "Data", "MIMII")
SCRIPTS_WAE_PATH = os.path.join(BASE_PROJECT_PATH, "WAE")
GLOBAL_PREPROCESSED_DATA_PATH = os.path.join(BASE_PROJECT_PATH, "Data", "global_preprocessed_notebook")
FOLD_SPECIFIC_TEMP_PATH = os.path.join(BASE_PROJECT_PATH, "evaluation_temp_notebook")
FINAL_RESULTS_PATH = os.path.join(BASE_PROJECT_PATH, "evaluation_results_notebook")

os.makedirs(GLOBAL_PREPROCESSED_DATA_PATH, exist_ok=True)
os.makedirs(FOLD_SPECIFIC_TEMP_PATH, exist_ok=True)
os.makedirs(FINAL_RESULTS_PATH, exist_ok=True)

In [95]:
# ----- evaluation config -----
N_FOLDS = 10 
N_BOOTSTRAP_SAMPLES = 1000 #  IC  ROC-AUC
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {DEVICE}")


Usando dispositivo: cuda


In [100]:
DEFAULT_ARGS_DICT = {
    "lr": 1e-3, "disc_lr": 1e-3, "epochs": 50, "weight_decay": 0,
    "critic_iterations": 5, "GP_hyperparam": 10.0, "WAE_regularization_term": 10.0,
    "dropout": 0.1, "embedding": 64, "hidden_dims": 30, "lstm_layers": 2,
    "batch_size": 32, "disc_hidden": 32, "disc_layers": 3,
    "tcn_hidden": 30, "tcn_layers": 10, "tcn_kernel": 3,
    "sparsity_weight": 1.0, "sparsity_parameter": 0.05, "nheads": 8,
    "feats": "all", # Será usado para construir nome de arquivo, mas number_features virá dos dados
    "NUMBER_FEATURES": 20, # Placeholder, será atualizado   
    "successive_iters": 10, "delta_worse": 0.02, "delta_better": 0.001,
    "model_name": "LSTMDiscriminator_TCN", # Ou o que for seu WAE-GAN principal
    "encoder_name": "TCN", "decoder_name": "TCN",
    "reconstruction_error_metric": "mse", "dtw_local_size": 5,
    "separate_comp": False, "init_loop": 0, "end_loop": 17, # Estes podem não ser relevantes para CV
    "force_training": True, # Para garantir que treine em cada fold
    "sensor": "tp2",
    "use_discriminator": True, # Para WAE-GAN
    "machine_type": "", "machine_id": "", # Preenchidos no loop
    # Adicionar os novos argumentos de override
    "input_train_data_path": None, "input_test_data_path": None,
    "output_model_base_path": None, "output_results_pkl_path": None,
    # Adicione quaisquer outros argumentos que seu ArgumentParser define com defaults
}

In [96]:
# MACHINE_CONFIGS = {
#     "fan": ["id_00", "id_02", "id_04", "id_06"],
#     "pump": ["id_00", "id_02", "id_04", "id_06"],
#     "slider": ["id_00", "id_02", "id_04", "id_06"],
#     "valve": ["id_00", "id_02", "id_04", "id_06"],
# }

MACHINE_CONFIGS = {
    "fan": ["id_00" ],
    "pump": ["id_00"],
    "slider": ["id_00"],
    "valve": ["id_00"],
}

In [97]:
# Célula 2: Função de Carga e Pré-processamento Global por Máquina
# (Usando files_train_test_split para obter caminhos e Array2Mfcc implicitamente via librosa)

# (Importações da Célula 1 já devem incluir Wav2Array, Array2Mfcc, e files_train_test_split)
import librosa # Para chamada direta a librosa.feature.mfcc

def get_all_mfcc_data_for_machine(machine_type, machine_id, num_mfcc_coeffs=20, target_sr=16000, force_reprocess=False):
    """
    Carrega ou pré-processa todos os dados MFCC para uma máquina/ID.
    Usa files_train_test_split para obter a lista de todos os caminhos de arquivo e seus rótulos.
    Salva/carrega de GLOBAL_PREPROCESSED_DATA_PATH para evitar reprocessamento.
    Retorna:
        X_all_tensors (list): Lista de tensores PyTorch (1, n_frames, n_coeffs).
        y_all_labels_original (np.array): Rótulos (Normal=1, Anormal=0, conforme mtsa.utils).
        actual_n_coeffs (int): Número real de coeficientes MFCC.
    """
    global_pkl_file_name = f"{machine_type}_{machine_id}_all_mfcc_data_c{num_mfcc_coeffs}.pkl"
    global_pkl_path = os.path.join(GLOBAL_PREPROCESSED_DATA_PATH, global_pkl_file_name)

    if not force_reprocess and os.path.exists(global_pkl_path):
        print(f"  Carregando dados MFCC pré-processados de: {global_pkl_path}")
        with open(global_pkl_path, "rb") as f:
            data = pkl.load(f)
        if data.get("num_mfcc_coeffs_requested") == num_mfcc_coeffs: # Checa se foi processado com os mesmos params
            return data["X_all_tensors"], data["y_all_labels_original"], data["actual_n_coeffs"]
        else:
            print(f"    Parâmetros de MFCC no arquivo ({data.get('num_mfcc_coeffs_requested')}) diferem do solicitado ({num_mfcc_coeffs}). Reprocessando.")

    print(f"  Processando arquivos .wav para {machine_type}/{machine_id} com {num_mfcc_coeffs} coeficientes MFCC...")
    id_path = os.path.join(MIMII_DATA_PATH, machine_type, machine_id)
    if not os.path.isdir(id_path):
        print(f"    Diretório não encontrado: {id_path}")
        return [], np.array([]), 0

    # files_train_test_split retorna X_train_files, X_test_files, y_train_labels, y_test_labels
    # Os rótulos são Normal=1, Anormal=0.
    # Para a CV, queremos todos os dados juntos primeiro.
    X_train_files, X_test_files, y_train_original, y_test_original = files_train_test_split(id_path, random_state=42) # random_state para consistência na divisão inicial se usada

    all_wav_files = np.concatenate((X_train_files, X_test_files))
    all_original_labels = np.concatenate((y_train_original, y_test_original))

    if len(all_wav_files) == 0:
        print(f"    Nenhum arquivo .wav encontrado ou combinado para {id_path}.")
        return [], np.array([]), 0

    wav_to_array_transformer = Wav2Array(sampling_rate=target_sr, mono=True)
    # Se você for usar sua classe Array2Mfcc, e ela não aceita n_mfcc,
    # ela usará o default do librosa (provavelmente 20).
    # Para ter controle, a chamada direta ao librosa.feature.mfcc é mais explícita.
    # array_to_mfcc_transformer = Array2Mfcc(sampling_rate=target_sr) # Se usar este, n_mfcc pode ser fixo.

    X_all_tensors = []
    valid_indices_for_labels = []
    actual_n_coeffs_detected = 0

    print(f"    Convertendo {len(all_wav_files)} arquivos .wav para arrays...")
    # É mais robusto processar WAVs um por um se alguns puderem estar corrompidos
    audio_arrays_list = []
    valid_original_indices_wav = []
    for idx, wav_file_path in enumerate(tqdm(all_wav_files, desc="      Processando WAVs")):
        try:
            # Wav2Array.transform espera uma lista de caminhos
            audio_array_single = wav_to_array_transformer.transform([wav_file_path])[0]
            if audio_array_single.ndim > 0 and audio_array_single.size > 0:
                audio_arrays_list.append(audio_array_single)
                valid_original_indices_wav.append(idx) # Guarda o índice do arquivo original que foi bem sucedido
        except Exception as e_single_wav:
            print(f"        Erro no arquivo {wav_file_path}: {e_single_wav}. Pulando arquivo.")
    
    if not audio_arrays_list:
         print(f"    Nenhum arquivo .wav pôde ser convertido para array.")
         return [], np.array([]), 0
    
    # Filtra os rótulos para corresponder apenas aos WAVs processados com sucesso
    all_original_labels_filtered = all_original_labels[valid_original_indices_wav]

    print(f"    Extraindo e normalizando MFCCs de {len(audio_arrays_list)} arrays de áudio...")
    for i, audio_array in enumerate(tqdm(audio_arrays_list, desc="      Processando MFCCs")):
        try:
            # Chamada direta ao librosa.feature.mfcc para controle de n_mfcc
            mfcc_cycle_raw = librosa.feature.mfcc(y=audio_array, sr=target_sr, n_mfcc=num_mfcc_coeffs)
            # mfcc_cycle_raw shape: (n_mfcc, n_frames)

            if mfcc_cycle_raw.shape[1] == 0: continue

            scaler = MinMaxScaler()
            mfcc_transposed_for_scaler = mfcc_cycle_raw.T # (n_frames, n_mfcc_coeffs)
            
            if mfcc_transposed_for_scaler.shape[0] == 0: continue
            if mfcc_transposed_for_scaler.ndim == 1:
                 mfcc_transposed_for_scaler = mfcc_transposed_for_scaler.reshape(1, -1)

            scaled_mfcc_transposed = scaler.fit_transform(mfcc_transposed_for_scaler)
            
            tensor_cycle = torch.tensor(scaled_mfcc_transposed, dtype=torch.float32).unsqueeze(0).to(DEVICE)
            # tensor_cycle shape: (1, n_frames, n_mfcc_coeffs)
            
            X_all_tensors.append(tensor_cycle)
            valid_indices_for_labels.append(i) # Índice relativo à audio_arrays_list

            if actual_n_coeffs_detected == 0 and tensor_cycle.shape[2] > 0:
                actual_n_coeffs_detected = tensor_cycle.shape[2]
                if actual_n_coeffs_detected != num_mfcc_coeffs:
                    print(f"    ALERTA: num_mfcc_coeffs solicitado={num_mfcc_coeffs}, detectado={actual_n_coeffs_detected}")

        except Exception as e:
            print(f"      Erro processando MFCC para um arquivo (índice {i} da lista de áudio): {e}. Pulando.")
            continue
            
    if not X_all_tensors:
        print(f"    Nenhum ciclo MFCC válido gerado para {machine_type}/{machine_id}.")
        return [], np.array([]), 0
    
    # Filtra os rótulos finais para corresponder apenas aos MFCCs que foram processados com sucesso
    y_all_labels_processed = all_original_labels_filtered[valid_indices_for_labels]

    data_to_save = {
        "X_all_tensors": [t.cpu() for t in X_all_tensors],
        "y_all_labels_original": y_all_labels_processed,
        "actual_n_coeffs": actual_n_coeffs_detected,
        "num_mfcc_coeffs_requested": num_mfcc_coeffs
    }
    with open(global_pkl_path, "wb") as f:
        pkl.dump(data_to_save, f)
    print(f"  Dados MFCC ({len(X_all_tensors)} ciclos) processados e salvos em: {global_pkl_path}")

    return X_all_tensors, y_all_labels_processed, actual_n_coeffs_detected

In [98]:
# Célula 3: Funções para Chamar Scripts .py e Manipular Dados de Fold

def save_fold_data_for_script(tensors_list, save_path):
    """Salva uma lista de tensores PyTorch em um arquivo .pkl."""
    # Os scripts esperam uma lista de tensores, onde cada tensor pode ser (1, seq_len, n_features)
    # Seus scripts `train_cycles_*.py` carregam com pkl.load() e depois convertem para device.
    # Esta função apenas salva a lista de tensores (que já estão no DEVICE correto se pré-processados no notebook).
    # No entanto, para manter a consistência com o que os scripts podem esperar de arquivos pkl
    # gerados pelo preprocessing_mimii.py, pode ser mais seguro salvar listas de arrays numpy
    # e deixar os scripts converterem para tensor e moverem para o device.
    # Mas se os scripts já lidam com tensores do device correto, está ok.

    # Vamos assumir que salvamos a lista de tensores diretamente.
    with open(save_path, "wb") as f:
        pkl.dump(tensors_list, f)

def run_script_for_fold(script_executable_path, args_for_script_dict):
    """
    Executa um script Python (e.g., train_cycles_adversarial.py) como um subprocesso.
    args_for_script_dict: dicionário de argumentos para o script.
    Retorna True se sucesso, False caso contrário.
    """
    cmd = [sys.executable, script_executable_path]
    for arg_name, arg_value in args_for_script_dict.items():
        if isinstance(arg_value, bool):
            if arg_value:
                cmd.append(f"--{arg_name}")  # Use underscore
        elif arg_value is not None:
            cmd.append(f"--{arg_name}")  # Use underscore
            cmd.append(str(arg_value))

    print(f"    Executando: {' '.join(cmd)}")
    try:
        # Timeout aumentado para 60 minutos (3600s) para treinos mais longos
        process = subprocess.run(cmd, capture_output=True, text=True, check=True, timeout=3600, encoding='utf-8', errors='replace')
        # print(f"      Output do Script (stdout):\n{process.stdout[-500:]}") # Últimas linhas
        # if process.stderr:
        #     print(f"      Output do Script (stderr):\n{process.stderr[-500:]}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"    ERRO ao executar script: {script_executable_path}")
        print(f"      Comando: {' '.join(e.cmd)}")
        print(f"      Código de Saída: {e.returncode}")
        print(f"      Stderr: {e.stderr}")
        print(f"      Stdout: {e.stdout}")
        return False
    except subprocess.TimeoutExpired as e:
        print(f"    TIMEOUT ({e.timeout}s) ao executar script: {script_executable_path}")
        print(f"      Comando: {' '.join(e.cmd)}")
        print(f"      Stderr (parcial): {e.stderr}")
        print(f"      Stdout (parcial): {e.stdout}")
        return False
    except Exception as e:
        print(f"    Exceção inesperada ao executar script {script_executable_path}: {e}")
        return False


def load_scores_from_script_output(expected_pkl_path):
    """
    Carrega os scores de anomalia do arquivo .pkl salvo pelo script de treino/predição.
    Espera que o .pkl contenha um dicionário com a chave "test" e dentro dela "reconstruction" e/ou "critic".
    """
    if not os.path.exists(expected_pkl_path):
        print(f"    Arquivo de scores não encontrado: {expected_pkl_path}")
        return None, None # reconstruction_errors, critic_scores

    try:
        with open(expected_pkl_path, "rb") as f:
            results_data = pkl.load(f)

        # Estrutura esperada baseada em train_cycles_adversarial.py:
        # results = {"test": {"reconstruction": reconstruction_errors, "critic": critic_scores},
        #            "train": {"reconstruction": args.train_reconstruction_errors, "critic": args.train_critic_scores}}
        # Para train_cycles.py (AE simples):
        # losses_over_time = {"test": test_losses, "train": args.train_losses} (test_losses são os erros de reconstrução)

        if "test" not in results_data:
            print(f"    Chave 'test' não encontrada no arquivo de scores: {expected_pkl_path}")
            return None, None

        test_results = results_data["test"]
        reconstruction_errors = np.array(test_results.get("reconstruction", []))
        critic_scores = np.array(test_results.get("critic", [])) # Pode ser vazio se AE simples

        # Se for AE simples, "test_losses" podem estar diretamente em results_data["test"]
        if not reconstruction_errors.any() and isinstance(test_results, list): # Caso do train_cycles.py
            reconstruction_errors = np.array(test_results)
            critic_scores = np.array([]) # Sem scores do crítico para AE simples

        if not reconstruction_errors.any():
             print(f"    Nenhum score de reconstrução encontrado em 'test' no arquivo: {expected_pkl_path}")
             return None, None

        return reconstruction_errors, critic_scores

    except Exception as e:
        print(f"    Erro ao carregar ou processar arquivo de scores {expected_pkl_path}: {e}")
        return None, None

# Função para determinar limiar ótimo (da resposta anterior, pode ir aqui)
def find_optimal_threshold_roc(y_true_binary_anomaly, anomaly_scores):
    fpr, tpr, thresholds = roc_curve(y_true_binary_anomaly, anomaly_scores)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    if optimal_threshold == np.inf or optimal_threshold == -np.inf or np.isnan(optimal_threshold):
        # print(f"    Aviso: Limiar ótimo da ROC foi {optimal_threshold}. Usando mediana.")
        return np.median(anomaly_scores)
    return optimal_threshold

In [ ]:
# Célula 4: Loop Principal de Validação Cruzada

all_machines_final_results = {}
# Diretório para dados temporários de cada fold (será recriado a cada execução da célula)
# Isso garante que não haja contaminação entre execuções do notebook.
if os.path.exists(FOLD_SPECIFIC_TEMP_PATH):
    shutil.rmtree(FOLD_SPECIFIC_TEMP_PATH)
os.makedirs(FOLD_SPECIFIC_TEMP_PATH, exist_ok=True)


for machine_type, machine_ids in MACHINE_CONFIGS.items():
    for machine_id in machine_ids:
        machine_name_id_str = f"{machine_type}_{machine_id}"
        print(f"\n===== Processando Máquina: {machine_name_id_str} =====")

        # 1. Carregar TODOS os dados MFCC (brutos, sem scaling ainda) para esta máquina/ID
        #    A função também salva/carrega de um cache global para evitar reprocessar .wavs
        X_all_tensors_raw, y_all_original_labels, detected_n_coeffs = get_all_mfcc_data_for_machine(
            machine_type, machine_id,
            num_mfcc_coeffs=DEFAULT_ARGS_DICT["NUMBER_FEATURES"], # Passa o N_MFCC desejado
            force_reprocess=False # Mude para True se quiser forçar o reprocessamento dos .wavs
        )

        if not X_all_tensors_raw:
            print(f"  Sem dados para {machine_name_id_str}. Pulando.")
            all_machines_final_results[machine_name_id_str] = {"error": "No data found or processed."}
            continue
        
        # Atualizar o número de features nos argumentos se foi detectado dinamicamente
        current_run_args_dict = DEFAULT_ARGS_DICT.copy()
        current_run_args_dict["NUMBER_FEATURES"] = detected_n_coeffs
        current_run_args_dict["machine_type"] = machine_type
        current_run_args_dict["machine_id"] = machine_id
        # Garantir que FEATS seja compatível com NUMBER_FEATURES para os scripts
        # Se NUMBER_FEATURES é, por exemplo, 20, FEATS pode ser "all" se for o default,
        # ou você pode ter uma lógica para mapear NUMBER_FEATURES para uma string FEATS válida.
        # Por simplicidade, se NUMBER_FEATURES é a fonte da verdade, FEATS é mais para nomeação.
        # Vamos assumir que os scripts usarão args.NUMBER_FEATURES diretamente se args.FEATS não levar a um mapeamento.

        # Rótulos para métricas: 1 para ANOMALIA, 0 para NORMAL
        # Seus rótulos originais: Normal=1, Anormal=0
        y_all_labels_anomaly_is_1 = 1 - y_all_original_labels

        if len(np.unique(y_all_labels_anomaly_is_1)) < 2:
            print(f"  Apenas uma classe presente para {machine_name_id_str} ({np.unique(y_all_original_labels)}). CV não é possível. Pulando.")
            all_machines_final_results[machine_name_id_str] = {"error": "Single class present."}
            continue

        fold_metrics_accumulator = { "roc_auc": [], "accuracy": [], "precision": [], "recall": [], "f1": [] }
        
        # Diretório temporário específico para esta máquina e seus folds
        machine_fold_temp_dir = os.path.join(FOLD_SPECIFIC_TEMP_PATH, machine_name_id_str)
        os.makedirs(machine_fold_temp_dir, exist_ok=True)

        skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
        # Usar np.arange pois X_all_tensors_raw é uma lista de tensores
        indices_for_kf = np.arange(len(X_all_tensors_raw))

        for fold_num, (train_indices, test_indices) in enumerate(tqdm(list(skf.split(indices_for_kf, y_all_labels_anomaly_is_1)), desc=f" Folds {machine_name_id_str}")):
            print(f"\n  --- Fold {fold_num + 1}/{N_FOLDS} para {machine_name_id_str} ---")
            
            fold_temp_dir_current = os.path.join(machine_fold_temp_dir, f"fold_{fold_num}")
            os.makedirs(fold_temp_dir_current, exist_ok=True)

            # Separar dados RAW do fold
            X_train_fold_raw = [X_all_tensors_raw[i] for i in train_indices]
            y_train_fold_original = y_all_original_labels[train_indices] # Normal=1
            
            X_test_fold_raw = [X_all_tensors_raw[i] for i in test_indices]
            y_test_fold_anomaly_is_1 = y_all_labels_anomaly_is_1[test_indices] # Anomalia=1

            # Isolar dados NORMAIS de treino do fold (RAW)
            X_train_normal_fold_raw = [
                X_train_fold_raw[i] for i, label in enumerate(y_train_fold_original) if label == 1
            ]

            if not X_train_normal_fold_raw:
                print(f"    Fold {fold_num + 1}: Sem dados normais para treino. Registrando NaNs.")
                for metric_list in fold_metrics_accumulator.values(): metric_list.append(np.nan)
                continue

            # Normalização DENTRO DO FOLD (Opção B)
            # Concatenar todos os frames dos dados normais de treino do fold para FITAR o scaler
            all_train_normal_frames_list = []
            for raw_tensor in X_train_normal_fold_raw: # raw_tensor é (1, seq_len, n_coeffs)
                all_train_normal_frames_list.append(raw_tensor.squeeze(0).cpu().numpy())
            
            combined_train_normal_frames = np.concatenate(all_train_normal_frames_list, axis=0) # (total_frames_normais_fold, n_coeffs)
            
            scaler_fold = MinMaxScaler()
            scaler_fold.fit(combined_train_normal_frames)

            # Aplicar scaler aos dados de treino normal do fold
            X_train_normal_fold_scaled_tensors = []
            for raw_tensor in X_train_normal_fold_raw:
                data_to_scale = raw_tensor.squeeze(0).cpu().numpy()
                scaled_data = scaler_fold.transform(data_to_scale)
                X_train_normal_fold_scaled_tensors.append(torch.tensor(scaled_data, dtype=torch.float32).unsqueeze(0).to(DEVICE))

            # Aplicar scaler aos dados de teste do fold
            X_test_fold_scaled_tensors = []
            for raw_tensor in X_test_fold_raw:
                data_to_scale = raw_tensor.squeeze(0).cpu().numpy()
                scaled_data = scaler_fold.transform(data_to_scale)
                X_test_fold_scaled_tensors.append(torch.tensor(scaled_data, dtype=torch.float32).unsqueeze(0).to(DEVICE))
            
            # Salvar dados SCALED do fold para o script usar
            fold_train_data_path_script = os.path.join(fold_temp_dir_current, "train_norm_scaled.pkl")
            save_fold_data_for_script(X_train_normal_fold_scaled_tensors, fold_train_data_path_script)
            
            fold_test_data_path_script = os.path.join(fold_temp_dir_current, "test_scaled.pkl")
            save_fold_data_for_script(X_test_fold_scaled_tensors, fold_test_data_path_script)

            # Configurar args para o script deste fold
            args_for_this_fold = current_run_args_dict.copy()
            args_for_this_fold["input_train_data_path"] = fold_train_data_path_script
            args_for_this_fold["input_test_data_path"] = fold_test_data_path_script
            args_for_this_fold["output_model_base_path"] = os.path.join(fold_temp_dir_current, "model_fold")
            args_for_this_fold["output_results_pkl_path"] = os.path.join(fold_temp_dir_current, "scores_fold.pkl")
            args_for_this_fold["epochs"] = DEFAULT_ARGS_DICT["epochs"] # Ou EPOCHS_CV
            # Garantir que os scripts não tentem carregar de caminhos fixos se os overrides são dados
            args_for_this_fold["data_folder"] = "./" # Para que os scripts não tentem construir a partir de "data/" global

            # Executar o script de treino/predição (WAE-GAN)
            script_to_run = os.path.join(SCRIPTS_WAE_PATH, "train_cycles_adversarial.py")
            success = run_script_for_fold(script_to_run, args_for_this_fold)

            if not success:
                print(f"    Falha ao executar script para o fold {fold_num + 1}. Registrando NaNs.")
                for metric_list in fold_metrics_accumulator.values(): metric_list.append(np.nan)
                continue

            # Carregar scores de anomalia salvos pelo script
            recon_errors, critic_sc = load_scores_from_script_output(args_for_this_fold["output_results_pkl_path"])

            if recon_errors is None or len(recon_errors) != len(y_test_fold_anomaly_is_1):
                print(f"    Scores de anomalia não carregados ou com tamanho incorreto para o fold {fold_num + 1}. Registrando NaNs.")
                for metric_list in fold_metrics_accumulator.values(): metric_list.append(np.nan)
                continue
            
            # Usar apenas erro de reconstrução como score de anomalia por enquanto
            # TODO: Incorporar critic_scores se a estratégia de combinação for definida
            anomaly_scores_fold = recon_errors

            # Calcular Métricas
            if len(np.unique(y_test_fold_anomaly_is_1)) < 2: # Checagem de segurança
                roc_auc, acc, prec, rec, f1 = 0.5 if np.all(y_test_fold_anomaly_is_1 == y_test_fold_anomaly_is_1[0]) else np.nan, np.nan, np.nan, np.nan, np.nan
            else:
                roc_auc = roc_auc_score(y_test_fold_anomaly_is_1, anomaly_scores_fold)
                optimal_thresh = find_optimal_threshold_roc(y_test_fold_anomaly_is_1, anomaly_scores_fold)
                y_pred_binary = (anomaly_scores_fold >= optimal_thresh).astype(int)
                acc = accuracy_score(y_test_fold_anomaly_is_1, y_pred_binary)
                prec = precision_score(y_test_fold_anomaly_is_1, y_pred_binary, zero_division=0)
                rec = recall_score(y_test_fold_anomaly_is_1, y_pred_binary, zero_division=0)
                f1 = f1_score(y_test_fold_anomaly_is_1, y_pred_binary, zero_division=0)

            fold_metrics_accumulator["roc_auc"].append(roc_auc)
            fold_metrics_accumulator["accuracy"].append(acc)
            fold_metrics_accumulator["precision"].append(prec)
            fold_metrics_accumulator["recall"].append(rec)
            fold_metrics_accumulator["f1"].append(f1)
            print(f"    Fold {fold_num + 1} Métricas: ROC-AUC={roc_auc:.4f}, F1={f1:.4f}, Acc={acc:.4f}")

        # Fim dos folds para a máquina/ID atual
        machine_summary_stats = {}
        valid_roc_aucs = [auc for auc in fold_metrics_accumulator["roc_auc"] if not np.isnan(auc)]
        if len(valid_roc_aucs) >= 2:
            mean_roc_auc = np.mean(valid_roc_aucs)
            bootstrap_means = [np.mean(resample(valid_roc_aucs, replace=True, n_samples=len(valid_roc_aucs), random_state=i)) for i in range(N_BOOTSTRAP_SAMPLES)]
            ci_lower = np.percentile(bootstrap_means, 2.5)
            ci_upper = np.percentile(bootstrap_means, 97.5)
            machine_summary_stats["roc_auc_mean"] = mean_roc_auc
            machine_summary_stats["roc_auc_ci"] = (ci_lower, ci_upper)
        else:
            machine_summary_stats["roc_auc_mean"] = np.nanmean(valid_roc_aucs) if valid_roc_aucs else np.nan
            machine_summary_stats["roc_auc_ci"] = (np.nan, np.nan)

        for metric_key in ["accuracy", "precision", "recall", "f1"]:
            valid_vals = [val for val in fold_metrics_accumulator[metric_key] if not np.isnan(val)]
            machine_summary_stats[f"{metric_key}_mean"] = np.mean(valid_vals) if valid_vals else np.nan
            machine_summary_stats[f"{metric_key}_std"] = np.std(valid_vals) if valid_vals else np.nan
        
        all_machines_final_results[machine_name_id_str] = machine_summary_stats
        print(f"  Resultados Consolidados para {machine_name_id_str}:")
        print(f"    ROC-AUC: {machine_summary_stats['roc_auc_mean']:.4f} (CI: [{machine_summary_stats['roc_auc_ci'][0]:.4f}-{machine_summary_stats['roc_auc_ci'][1]:.4f}])")
        print(f"    F1-Score: {machine_summary_stats.get('f1_mean', np.nan):.4f} +/- {machine_summary_stats.get('f1_std', np.nan):.4f}")


# Salvar todos os resultados finais
final_pkl_path = os.path.join(FINAL_RESULTS_PATH, "all_machines_evaluation_summary.pkl")
with open(final_pkl_path, "wb") as f:
    pkl.dump(all_machines_final_results, f)
print(f"\nResultados finais de todas as máquinas salvos em: {final_pkl_path}")

# Limpar a pasta temporária principal após a conclusão de todas as máquinas
# shutil.rmtree(FOLD_SPECIFIC_TEMP_PATH)
# print(f"Pasta temporária {FOLD_SPECIFIC_TEMP_PATH} removida.")


===== Processando Máquina: fan_id_00 =====
  Carregando dados MFCC pré-processados de: C:\Users\igorc\Desktop\Implementação TCC\Data\global_preprocessed_notebook\fan_id_00_all_mfcc_data_c20.pkl


 Folds fan_id_00:   0%|          | 0/10 [00:00<?, ?it/s]


  --- Fold 1/10 para fan_id_00 ---


KeyError: 'EPOCHS'

In [ ]:
# Célula 5: Geração das Tabelas LaTeX
# (Reutilizar a função generate_latex_tables(all_machines_final_results) da resposta anterior)
# Ex:
# generate_latex_tables(all_machines_final_results)
# (Você pode carregar do .pkl se executar esta célula separadamente)
# with open(os.path.join(FINAL_RESULTS_PATH, "all_machines_evaluation_summary.pkl"), "rb") as f:
#    loaded_results = pkl.load(f)
# generate_latex_tables(loaded_results)